In [1]:
import numpy as np
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearnex import patch_sklearn, config_context
patch_sklearn()
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification, make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict, KFold, train_test_split
from sklearn.metrics import average_precision_score, precision_recall_curve, multilabel_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay, classification_report,PrecisionRecallDisplay,accuracy_score, f1_score, precision_score, recall_score

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
import seaborn as sns

import matplotlib.pyplot as plt
import plotly.express as px

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
past = 'DataSet/Final/DataSet2017_2021.csv'
df = pd.read_csv(past, delimiter = ';',  low_memory=False, encoding='UTF-8', index_col = False)

In [3]:
df1 = df.copy()
#df1.columns
df1 = df1[(df1['ano'] == 2020) | (df1['ano'] == 2019)]
df1 = df1.reset_index(drop=True)
df1

,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,...,veiculos,latitude,longitude,regional,delegacia,uop,ano,mes,dia,mortes
0,2019-01-01,Tuesday,01:30:00,SP,116,218.0,Guarulhos,Falta De Atencao A Conducao,Colisao Com Objeto Estatico,Com Vitimas Feridas,...,1,-23.460520,-46.487725,Sr-Sp,Del6/1,Uop01/Sp,2019,January,1,0
1,2019-01-01,Tuesday,01:30:00,PR,373,177.3,Ponta Grossa,Falta De Atencao A Conducao,Colisao Traseira,Com Vitimas Feridas,...,1,-25.055340,-50.227768,Sr-Pr,Del7/3,Uop01/Pr,2019,January,1,0
2,2019-01-01,Tuesday,00:45:00,SC,101,16.0,Garuva,Animais Na Pista,Colisao Com Objeto Estatico,Com Vitimas Feridas,...,1,-26.121600,-48.882600,Sr-Sc,Del8/3,Uop01/Sc,2019,January,1,0
3,2019-01-01,Tuesday,00:00:00,CE,20,429.5,Fortaleza,Ingestao De Substancias Psicoativas,Colisao Com Objeto Estatico,Com Vitimas Feridas,...,1,-3.769990,-38.670063,Sr-Ce,Del16/1,Uop02/Ce,2019,January,1,0
4,2019-01-01,Tuesday,01:00:00,MG,381,327.0,Nova Era,Falta De Atencao A Conducao,Colisao Transversal,Com Vitimas Feridas,...,2,-19.760900,-43.030600,Sr-Mg,Del4/3,Uop01/Mg,2019,January,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126896,2020-11-27,Friday,20:30:00,ES,101,287.0,Cariacica,Falta De Atencao A Conducao,Colisao Lateral,Com Vitimas Feridas,...,2,-20.264935,-40.384717,Sr-Es,Uop01,Del01,2020,November,27,0
126897,2020-12-18,Friday,03:20:00,RO,364,1071.0,Porto Velho,Desobediencia As Normas De Transito Pelo Condutor,Colisao Frontal,Com Vitimas Fatais,...,2,-9.752286,-66.615171,Sr-Ro,Uop04,Del01,2020,December,18,1
126898,2020-08-15,Saturday,18:30:00,RS,116,156.6,Caxias Do Sul,Desobediencia As Normas De Transito Pelo Condutor,Colisao Lateral,Com Vitimas Feridas,...,2,-29.207310,-51.177579,Sr-Rs,Uop01,Del05,2020,August,15,0
126899,2020-12-30,Wednesday,11:00:00,PI,343,20.0,Parnaiba,Falta De Atencao A Conducao,Colisao Transversal,Com Vitimas Feridas,...,2,-2.981128,-41.762501,Sr-Pi,Uop01,Del05,2020,December,30,0


In [4]:
df1['uf_br'] = df1['uf'] + '_' + df1['br'].map(str)
df1['dia_mes'] = df1['dia'].map(str) + ' ' + df1['mes'].map(str)

In [5]:
Categoria01 = ['Fenomenos Da Natureza','Chuva', 'Acumulo De Agua Sobre O Pavimento', 'Neblina',
               'Demais Fenomenos Da Natureza', 'Fumaca', 'Animais Na Pista', 'Pista Escorregadia',
               'Acumulo De Oleo Sobre O Pavimento', 'Acumulo De Areia Ou Detritos Sobre O Pavimento']

Categoria02 = ['Falta De Atencao A Conducao', 'Ausencia De Reacao Do Condutor', 'Reacao Tardia Ou Ineficiente Do Condutor', 
               'Manobra De Mudanca De Faixa',  'Condutor Dormindo', 'Falta De Atencao Do Pedestre', 
               'Acessar A Via Sem Observar A Presenca Dos Outros Veiculos',  'Entrada Inopinada Do Pedestre', 'Frear Bruscamente', 
               'Deixar De Acionar O Farol Da Motocicleta (Ou Similar)', 'Mal Subito', 'Mal Subito Do Condutor', 'Agressao Externa']

Categoria03 = ['Avarias E/Ou Desgaste Excessivo No Pneu', 'Defeito Mecanico No Veiculo', 
               'Deficiencia Ou Nao Acionamento Do Sistema De Iluminacao/Sinalizacao Do Veiculo',
               'Demais Falhas Mecanicas Ou Eletricas', 'Farois Desregulados', 'Iluminacao Deficiente', 
               'Problema Com O Freio', 'Problema Na Suspensao']

Categoria04 = ['Ingestao De Alcool', 'Ingestao De Substancias Psicoativas', 
               'Ingestao De Alcool E/Ou Substancias Psicoativas Pelo Pedestre', 'Ingestao De Alcool Pelo Condutor',  
               'Ingestao De Substancias Psicoativas Pelo Condutor', 'Ingestao De Alcool Ou De Substancias Psicoativas Pelo Pedestre']

Categoria05 = [ 'Desobediencia As Normas De Transito Pelo Condutor' , 'Ultrapassagem Indevida', 
               'Desobediencia As Normas De Transito Pelo Pedestre', 'Transitar Na Contramao', 
               'Pedestre Andava Na Pista', 'Desrespeitar A Preferencia No Cruzamento', 
               'Condutor Desrespeitou A Iluminacao Vermelha Do Semaforo',
               'Condutor Deixou De Manter Distancia Do Veiculo Da Frente', 'Estacionar Ou Parar Em Local Proibido',
               'Condutor Usando Celular', 'Transitar No Acostamento', 'Trafegar Com Motocicleta (Ou Similar) Entre As Faixas',
               'Participar De Racha', 'Velocidade Incompativel', 'Nao Guardar Distancia De Seguranca', 'Conversao Proibida', 
               'Acesso Irregular', 'Retorno Proibido', 'Modificacao Proibida', 'Pedestre Cruzava A Pista Fora Da Faixa', 
               'Carga Excessiva E/Ou Mal Acondicionada', 'Objeto Estatico Sobre O Leito Carrocavel', 'Transitar Na Calcada']

Categoria06 = [ 'Sinalizacao Da Via Insuficiente Ou Inadequada', 'Falta De Acostamento', 'Defeito Na Via', 'Demais Falhas Na Via',
               'Afundamento Ou Ondulacao No Pavimento', 'Pista Esburacada', 'Acostamento Em Desnivel', 'Sinalizacao Mal Posicionada',       
               'Deficiencia Do Sistema De Iluminacao/Sinalizacao', 'Ausencia De Sinalizacao', 
               'Falta De Elemento De Contencao Que Evite A Saida Do Leito Carrocavel', 'Sinalizacao Encoberta',
               'Faixas De Transito Com Largura Insuficiente', 'Semaforo Com Defeito', 'Restricao De Visibilidade Em Curvas Ver', 
               'Restricao De Visibilidade Em Curvas Horizontais', 'Pista Em Desnivel', 'Restricao De Visibilidade', 'Curva Acentuada',       
               'Declive Acentuado', 'Redutor De Velocidade Em Desacordo', 'Obras Na Pista', 'Obstrucao Na Via', 
               'Area Urbana Sem A Presenca De Local Apropriado Para A Travessia De Pedestres', 'Desvio Temporario', 
               'Restricao De Visibilidade Em Curvas Verticais']



In [6]:
df2 = df1.copy()
df2['causa_acidente'] = df2['causa_acidente'].replace(Categoria01, 'categoria 01')
df2['causa_acidente'] = df2['causa_acidente'].replace(Categoria02, 'categoria 02')
df2['causa_acidente'] = df2['causa_acidente'].replace(Categoria03, 'categoria 03')
df2['causa_acidente'] = df2['causa_acidente'].replace(Categoria04, 'categoria 04')
df2['causa_acidente'] = df2['causa_acidente'].replace(Categoria05, 'categoria 05 ')
df2['causa_acidente'] = df2['causa_acidente'].replace(Categoria06, 'categoria 06')
df2['condicao_metereologica'] = df2['condicao_metereologica'].replace(['Garoa/Chuvisco', 'Nublado', 'Chuva'], 'Chuca/Nublado')
df2['condicao_metereologica'] = df2['condicao_metereologica'].replace(['Ceu Claro','Sol'], 'Ceu Claro/Sol')

df2.columns

Index(['data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio',
       'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia',
       'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via',
       'uso_solo', 'pessoas', 'mortos', 'feridos_leves', 'feridos_graves',
       'ilesos', 'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude',
       'regional', 'delegacia', 'uop', 'ano', 'mes', 'dia', 'mortes', 'uf_br',
       'dia_mes'],
      dtype='object')

In [8]:
def resultados(y_true, y_pred, y_score, cl):
    
    Y_true = pd.get_dummies(y_true).values
    Y_pred = pd.get_dummies(y_pred).values
    Y_score = np.array(y_score)

    precision = dict()
    recall = dict()
    average_precision = dict()

    for i,j in enumerate(label):
        precision[i], recall[i], _ = precision_recall_curve(Y_true[:, i], Y_score[:, i])
        average_precision[i] = average_precision_score(Y_true[:, i], Y_score[:, i])
    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(Y_true.ravel(), Y_score.ravel())
    average_precision["micro"] = average_precision_score(Y_true, Y_score, average="micro")

    prec = precision["micro"]
    reca = recall["micro"]
    aver = average_precision["micro"]


    pscore = precision_score(y_true, y_pred, average = 'macro')
    rscore = recall_score(y_true, y_pred, average = 'macro')
    fscore = f1_score(y_true, y_pred, average = 'macro') 
    ascore = accuracy_score(y_true, y_pred)

    report.append(classification_report(y_true, y_pred, target_names=label))

    disp = ConfusionMatrixDisplay.from_predictions(y_true,y_pred,display_labels=label,
                                                   cmap=plt.cm.Blues,xticks_rotation = 45,normalize=norm)
    plt.savefig('Figuras/DecisionTreeClassifier.jpg')
    plt.show()
    
    np.savetxt(past + 'Resultados/' + cl + 'core.txt',[pscore, rscore, fscore, ascore], delimiter=',')
    np.savetxt(past + 'Resultados/' + cl + 'Prec.txt', prec, delimiter=',')
    np.savetxt(past + 'Resultados/' + cl + 'Reca.txt', reca, delimiter=',')
    np.savetxt(past + 'Resultados/' + cl + 'Aver.txt', [aver], delimiter=',')
    
return (prec, reca, aver, pscore, rscore, fscore, ascore)


SyntaxError: 'return' outside function (Temp/ipykernel_13276/1583119384.py, line 40)

In [9]:
colc = ['mes','causa_acidente', 'fase_dia', 'condicao_metereologica']
colr = 'classificacao_acidente'
coln = ['latitude', 'longitude', 'dia', 'veiculos', 'pessoas']
col = []
col.extend(coln)
col.extend(colc)

df3 = pd.DataFrame()
df3[colr]= df2[[colr]]

df3[colc] = df2[colc]
df3[coln] = df2[coln]

In [10]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X = df3.drop(colr, axis = 1)
X = pd.get_dummies(X[)
feature_names = X.columns
y = df3[colr]
#y = pd.get_dummies(y)
label = df3[colr].unique()


In [11]:
nr = NearMiss()
#smt = SMOTE()
X, y = nr.fit_resample(X, y)
X = X.values
y = y.values

In [12]:
norm = 'true'
n = 10
rs = 0
kf = KFold(n_splits=n)


result = []


In [13]:
clf = DecisionTreeClassifier(random_state=rs)
y_true = []
y_pred = []
y_score = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred.extend(clf.predict(X_test))
    y_true.extend(y_test)
    y_score.extend(clf.predict_proba(X_test))

reult.append(resultados(y_true, y_pred, y_score, 'DecisionTreeClassifier'))

NameError: name 'reult' is not defined

In [ ]:
clf = GradientBoostingClassifier(random_state=rs)
y_true = []
y_pred = []
y_score = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred.extend(clf.predict(X_test))
    y_true.extend(y_test)
    y_score.extend(clf.predict_proba(X_test))
    
reult.append(resultados(y_true, y_pred, y_score, 'GradientBoostingClassifier'))

In [ ]:
clf = MLPClassifier(random_state=rs, max_iter = 500)
y_true = []
y_pred = []
y_score = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred.extend(clf.predict(X_test))
    y_true.extend(y_test)
    y_score.extend(clf.predict_proba(X_test))
    
    
reult.append(resultados(y_true, y_pred, y_score, 'MLPClassifier'))

In [14]:
clf = svm.SVC(random_state=rs, probability = True, kernel ='sigmoid')
y_true = []
y_pred = []
y_score = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred.extend(clf.predict(X_test))
    y_true.extend(y_test)
    y_score.extend(clf.predict_proba(X_test))

reult.append(resultados(y_true, y_pred, y_score, 'svm'))

NameError: name 'reult' is not defined

In [ ]:
clf = GaussianNB()
y_true = []
y_pred = []
y_score = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred.extend(clf.predict(X_test))
    y_true.extend(y_test)
    y_score.extend(clf.predict_proba(X_test))

reult.append(resultados(y_true, y_pred, y_score, 'GaussianNB'))

In [ ]:
clf = RandomForestClassifier(random_state=100, max_depth=2)
y_true = []
y_pred = []
y_score = []
importances = 0
std = 0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred.extend(clf.predict(X_test))
    y_true.extend(y_test)
    y_score.extend(clf.predict_proba(X_test))
    importances = importances + clf.feature_importances_
    std = std + np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
  

reult.append(resultados(y_true, y_pred, y_score, 'RandomForestClassifier'))

In [ ]:
importances = importances/n
std = std/n
feature_names = pd.get_dummies(df3.drop(colr, axis = 1), prefix='', prefix_sep='')
forest_importances = pd.DataFrame(np.array([importances, std, feature_names.columns]).T, 
                                  columns = ['Media', 'Desvio Padrão', 'Atritutos'] )
fig = px.bar(forest_importances, x = 'Atritutos', y = 'Media', error_y = 'Desvio Padrão', 
             title = 'Importância do recurso com base na diminuição média da impureza')
fig.show()

fig.write_image("Figuras/importancia.jpeg")

In [ ]:
classificadores = ['Decision Treem', 'Gradient Boosted Tree', 'Perceptron multicamadas', 'Support Vector Machine', 'Naive Bayes', 'Random Forest']
reca = 
for i,j in enumerate(classificadores):         
    plt.plot(reca[i], prec[i], label = j)
plt.title("Micro Média em Todas as Classes")
plt.xlabel('Sensibilidade')
plt.ylabel('Precisão')
plt.legend(loc="upper right")
plt.savefig('Figuras/PrecisaoSensibilidade.jpg')
plt.show()


In [ ]:
metricas = ['Precisão %', 'Sensibilidade %', 'F1 Score %', 'Acurácia %']
tab = pd.DataFrame(np.array([pscore, rscore, fscore, ascore])*100, columns = classificadores, index = metricas)
tab.index.name = 'Metricas'
tab = tab.round()


In [ ]:
print(tab.T.to_latex(index=True, longtable = False))

In [ ]:
print(df3.head().to_latex(index=True))

In [ ]:
df3.head()